# **LOAD GOOGLE DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install speechrecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 5.7 MB/s eta 0:00:00


In [ ]:
!pip -q install gtts

In [ ]:
# Import Libraries
import json
import nltk
import time
import random
import string
import pickle #untuk menyimpan dan membaca data ke dalam atau dari suatu file berformat
import numpy as np
import pandas as pd
from gtts import gTTS 
from io import BytesIO #dapat disimpan sebagai byte dalam buffer dalam memori saat kita menggunakan operasi Byte IO modul io.
import tensorflow as tf
import IPython.display as ipd
import speech_recognition as sr 
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Model
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Flatten, Dense, GlobalMaxPool1D

# **Download NLTK Package**

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Importing the dataset
with open('/content/drive/MyDrive/Chatbot/Dataset/dataset.json') as content:
  data1 = json.load(content)

# Mendapatkan semua data ke dalam list
tags = [] # data tag
inputs = [] # data input atau pattern
responses = {} # data respon
words = [] # Data kata 
classes = [] # Data Kelas atau Tag
documents = [] # Data Kalimat Dokumen
ignore_words = ['?', '!'] # Mengabaikan tanda spesial karakter

for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['patterns']:
    inputs.append(lines)
    tags.append(intent['tag'])
    for pattern in intent['patterns']:
      w = nltk.word_tokenize(pattern)
      words.extend(w)
      documents.append((w, intent['tag']))
      # add to our classes list
      if intent['tag'] not in classes:
        classes.append(intent['tag'])

# Konversi data json ke dalam dataframe
data = pd.DataFrame({"patterns":inputs, "tags":tags})

# **Menghapus Punktuasi**

Tahapan praproses pada data teks yang pertama adalah menghapus punktuasi atau tanda baca seperti special character yaitu '!' (tanda seru) ',' (tanda koma) '.' (tanda titik sebagai berhenti) '?' (tanda tanya) dan tanda baca yang lain. Tahapan ini gunanya untuk mempermudah pemrosesan data teks yang akan kita olah.

In [ ]:
# Removing Punctuations (Menghilangkan Punktuasi)
data['patterns'] = data['patterns'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['patterns'] = data['patterns'].apply(lambda wrd: ''.join(wrd))
data

,patterns,tags
0,hallo,greeting
1,hai,greeting
2,halo,greeting
3,hei,greeting
4,hi,greeting
...,...,...
70,apa itu ukm,ukm
71,ukm apa sih,ukm
72,ukm,ukm
73,ukm adalah,ukm


# **Lematisasi**

Lematisasi atau Lemmatization adalah proses dimana merujuk pada melakukan sesuatu menggunakan vocabulary atau kosakata dan analisis morfologi kata-kata untuk menghilangkan inflectional endings only dan untuk mengembalikan bentuk dictionary (kata dalam kamus) dari sebuah kata yang dikenal sebagai *lemma*.

In [ ]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

print (len(words), "unique lemmatized words", words)

83 unique lemmatized words [',', 'ada', 'adalah', 'afternoon', 'aja', 'apa', 'apakah', 'apasih', 'aplikasi', 'bai', 'banyak', 'berlangsung', 'besok', 'bimbingan', 'bot-ol', 'botol', 'bro', 'buat', 'bulan', 'bye', 'byee', 'chatbot', 'dadah', 'dah', 'di', 'diciptakan', 'good', 'hai', 'hallo', 'halo', 'hari', 'hei', 'hi', 'hy', 'ini', 'ittp', 'itu', 'jadwal', 'jumpa', 'kampus', 'kamu', 'kapan', 'kasih', 'kawan', 'kelasku', 'keluar', 'kita', 'kok', 'ku', 'kuliah', 'makasih', 'malam', 'mata', 'mesin', 'morning', 'nlp', 'pagi', 'pembuatmu', 'pemrograman', 'penciptamu', 'saja', 'sampai', 'saya', 'see', 'sekarang', 'selamat', 'si', 'siang', 'siapa', 'sih', 'sore', 'tentang', 'terima', 'thank', 'thanks', 'tinggal', 'uas', 'ukm', 'untuk', 'ut', 'ya', 'yang', 'you']


# **Menyortir Data Kelas Tags**

In [ ]:
# sort classes
classes = sorted(list(set(classes)))
print (len(classes), "classes", classes)

11 classes ['bimbingan', 'bot-ol', 'goodbye', 'greeting', 'jadwal_uas', 'jadwal_uts', 'pencipta_bot-ol', 'penjelasan_jadwal_kuliah', 'tentang_kampus', 'terimakasih', 'ukm']


# **Mencari Jumlah Keseluruhan Data Teks**

In [ ]:
# documents = combination between patterns and intents
print (len(documents), "documents")

629 documents


# **Tokenisasi**

Setelah proses lematisasi dan mencari tahu data classes dan jumlah keseluruhan data patterns dengan intents-nya. Maka, tahapan selanjutnya proses tokenisasi. Apa itu Tokenisasi?

Tokenisasi adalah suatu proses memberikan urutan karakter dan sebuah unit dokumen terdefinisi. Tokenisasi juga merupakan tugas untuk memecah kalimat menjadi bagian-bagian yang disebut dengan 'Token' dan menghilangkan bagian tertentu seperti tanda baca.

In [ ]:
# Tokenize the data (Tokenisasi Data)
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['patterns'])
train = tokenizer.texts_to_sequences(data['patterns'])
train

[[32],
 [33],
 [34],
 [35],
 [36],
 [37],
 [38],
 [39],
 [40],
 [41],
 [42],
 [43],
 [44],
 [45],
 [46],
 [2, 6, 4],
 [9, 4],
 [4, 6, 2],
 [9, 5, 4],
 [4, 14],
 [47, 15, 4],
 [48, 49, 15, 4, 6],
 [20, 50, 4],
 [9, 51],
 [4, 52, 20, 16, 9, 5],
 [9, 21, 4],
 [16, 53, 22, 5, 4],
 [9, 21],
 [16, 54, 55, 2],
 [56],
 [57],
 [23],
 [58],
 [59, 23],
 [60, 61],
 [62, 63],
 [64],
 [65, 24],
 [66],
 [67, 24],
 [25, 26],
 [68],
 [25, 26, 69],
 [3, 1, 7, 8, 27, 28],
 [1, 27, 28],
 [1, 17, 2],
 [3, 1, 7, 8, 18],
 [1, 7, 8, 70, 71],
 [17, 72, 2, 73, 19],
 [1, 7, 8, 18, 6, 3, 5],
 [74, 1, 7, 8, 29, 2],
 [1, 7, 8, 18],
 [17, 1, 7, 8, 29, 2, 19],
 [3, 1, 12, 30],
 [1, 12, 3],
 [12],
 [3, 12],
 [12, 31, 3, 5],
 [3, 1, 10, 30],
 [1, 10, 3],
 [10],
 [3, 10],
 [10, 31, 3, 5],
 [1, 10, 3, 75, 76, 2],
 [15, 11, 6],
 [11, 6, 2],
 [11],
 [2, 5, 11, 6],
 [11, 14],
 [1, 11, 77, 3, 19],
 [2, 6, 13],
 [13, 2, 5],
 [13],
 [13, 14],
 [22, 2, 78, 5, 13, 79, 80, 81]]

# **Padding**

Padding adalah Suatu proses untuk mengubah setiap sequence agar memiliki panjang yang sama. Pada padding, setiap sequence dibuat sama panjang dengan menambahkan nilai 0 secara suffiks atau prefiks hingga mencapai panjang maksimum sequence. Selain itu padding juga dapat memotong sequence hingga panjangnya sesuai dengan panjang maksimum sequence.

Padding juga adalah proses untuk membuat setiap kalimat pada teks memiliki panjang yang seragam. Sama seperti melakukan resize gambar, agar resolusi setiap gambar sama besar. Untuk menggunakan padding bisa impor library pad_sequence. Kemudian buat panggil fungsi pad_sequence() dan masukkan sequence hasil tokenisasi sebagai parameternya.



In [ ]:
# Apply padding 
x_train = pad_sequences(train)
print(x_train) # Padding Sequences

[[ 0  0  0  0  0  0  0 32]
 [ 0  0  0  0  0  0  0 33]
 [ 0  0  0  0  0  0  0 34]
 [ 0  0  0  0  0  0  0 35]
 [ 0  0  0  0  0  0  0 36]
 [ 0  0  0  0  0  0  0 37]
 [ 0  0  0  0  0  0  0 38]
 [ 0  0  0  0  0  0  0 39]
 [ 0  0  0  0  0  0  0 40]
 [ 0  0  0  0  0  0  0 41]
 [ 0  0  0  0  0  0  0 42]
 [ 0  0  0  0  0  0  0 43]
 [ 0  0  0  0  0  0  0 44]
 [ 0  0  0  0  0  0  0 45]
 [ 0  0  0  0  0  0  0 46]
 [ 0  0  0  0  0  2  6  4]
 [ 0  0  0  0  0  0  9  4]
 [ 0  0  0  0  0  4  6  2]
 [ 0  0  0  0  0  9  5  4]
 [ 0  0  0  0  0  0  4 14]
 [ 0  0  0  0  0 47 15  4]
 [ 0  0  0 48 49 15  4  6]
 [ 0  0  0  0  0 20 50  4]
 [ 0  0  0  0  0  0  9 51]
 [ 0  0  4 52 20 16  9  5]
 [ 0  0  0  0  0  9 21  4]
 [ 0  0  0 16 53 22  5  4]
 [ 0  0  0  0  0  0  9 21]
 [ 0  0  0  0 16 54 55  2]
 [ 0  0  0  0  0  0  0 56]
 [ 0  0  0  0  0  0  0 57]
 [ 0  0  0  0  0  0  0 23]
 [ 0  0  0  0  0  0  0 58]
 [ 0  0  0  0  0  0 59 23]
 [ 0  0  0  0  0  0 60 61]
 [ 0  0  0  0  0  0 62 63]
 [ 0  0  0  0  0  0  0 64]
 

# **Encoding Text**

Encoding merupakan suatu konversi atau pengkodean yang dimana data kategorik seperti huruf atau data teks menjadi data numerik atau angka menyesuaikan dengan data label yang digunakan. Pada proses tahapan ini, encoding mengubah data teks pada kolom data tags menjadi data numerik dengan bahasa biner komputer yaitu 0 dan 1.

Tujuan dari encoding ini adalah mempermudah saat proses komputasi data teks dan modelling.

In [ ]:
# Encoding the outputs 
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [ ]:
print(y_train) #Label Encodings

[ 3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  1  1  1  1  1  1  8  8  6
  6  6  6  6  6  2  2  2  2  2  2  2  2  2  9  9  9  9  9  7  7  7  7  7
  7  7  7  7  7  5  5  5  5  5  4  4  4  4  4  4  0  0  0  0  0  0 10 10
 10 10 10]


Tokenizer pada Tensorflow memberikan token unik untuk setiap kata yang berbeda. Dan juga padding dilakukan untuk mendapatkan semua data dengan panjang yang sama sehingga dapat mengirimkannya ke lapisan atau layer RNN. variabel target juga dikodekan menjadi nilai desimal.

# **Input Length, Output Length and Vocabulary**

In [ ]:
# input length
input_shape = x_train.shape[1]
print(input_shape)

8


In [ ]:
# define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ", vocabulary)

# output length
output_length = le.classes_.shape[0]
print("output length: ", output_length)

number of unique words :  81
output length:  11


In [ ]:
pickle.dump(words, open('/content/drive/MyDrive/Chatbot/Model3/words.pkl','wb'))
pickle.dump(classes, open('/content/drive/MyDrive/Chatbot/Model3/classes.pkl','wb'))

Input length dan output length terlihat sangat jelas hasilnya. Mereka adalah untuk bentuk input dan bentuk output dari jaringan syaraf pada algoritma Neural Network.

Vocabulary Size adalah untuk lapisan penyematan untuk membuat representasi vektor unik untuk setiap kata.

# **Neural Network Model**

Jaringan syaraf yang terdiri dari lapisan embedding yang merupakan salah satu hal yang paling kuat di bidang pemrosesan bahasa alami atau NLP. output atau keluaran dari lapisan embedding adalah input dari lapisan berulang (recurrent) dengan LSTM gate. Kemudian, output diratakan dan lapisan Dense digunakan dengan fungsi aktivasi softmax.

Bagian utama adalah lapisan embedding yang memberikan vektor yang sesuai untuk setiap kata dalam dataset.

In [ ]:
# Creating the model (Membuat Modeling)
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i) # Layer Embedding
x = LSTM(10, return_sequences=True)(x) # Layer Long Short Term Memory
x = Flatten()(x) # Layer Flatten
x = Dense(output_length, activation="softmax")(x) # Layer Dense
model  = Model(i,x)

# Compiling the model (Kompilasi Model)
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary() # Menampilkan Parameter Model

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 embedding (Embedding)       (None, 8, 10)             820       
                                                                 
 lstm (LSTM)                 (None, 8, 10)             840       
                                                                 
 flatten (Flatten)           (None, 80)                0         
                                                                 
 dense (Dense)               (None, 11)                891       
                                                                 
Total params: 2,551
Trainable params: 2,551
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Training the model (Latih Model Data)
train = model.fit(x_train, y_train, epochs=400)

Epoch 1/400
3/3 [==============================] - 3s 11ms/step - loss: 2.3973 - accuracy: 0.0667
Epoch 2/400
3/3 [==============================] - 0s 11ms/step - loss: 2.3932 - accuracy: 0.1467
Epoch 3/400
3/3 [==============================] - 0s 9ms/step - loss: 2.3891 - accuracy: 0.1467
Epoch 4/400
3/3 [==============================] - 0s 10ms/step - loss: 2.3856 - accuracy: 0.1600
Epoch 5/400
3/3 [==============================] - 0s 10ms/step - loss: 2.3812 - accuracy: 0.1600
Epoch 6/400
3/3 [==============================] - 0s 9ms/step - loss: 2.3769 - accuracy: 0.1600
Epoch 7/400
3/3 [==============================] - 0s 10ms/step - loss: 2.3720 - accuracy: 0.1600
Epoch 8/400
3/3 [==============================] - 0s 10ms/step - loss: 2.3666 - accuracy: 0.1600
Epoch 9/400
3/3 [==============================] - 0s 11ms/step - loss: 2.3607 - accuracy: 0.1467
Epoch 10/400
3/3 [==============================] - 0s 10ms/step - loss: 2.3548 - accuracy: 0.1467
Epoch 11/400
3/3 [===

In [ ]:
model.save('/content/drive/MyDrive/Chatbot/Model3/chat_model.h5')

print('Model Created Successfully!')

Model Created Successfully!


# **Testing Chatbot**

Setelah mengetahui hasil dari akurasi dan loss pada model yang telah ditetapkan dengan algoritma Neural Network dan LSTM. Maka, tahapan selanjutnya adalah menguji atau testing pada chatbot yang telah dilatih sebelumnya dan melihat apakah sesuai atau tidak pada saat kita masukan teks kalimat pertanyaannya.

Pada pengujian ini menggunakan metode input atau masukan sesuai dengan kalimat pertanyaan yang dimasukkan.

In [ ]:
# Membuat Input Chat
while True:
  texts_p = []
  prediction_input = input('Kamu : ')
  
  # Menghapus punktuasi dan konversi ke huruf kecil
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)

  # Tokenisasi dan Padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)

  # Mendapatkan hasil keluaran pada model 
  output = model.predict(prediction_input)
  output = output.argmax()

  # Menemukan respon sesuai data tag dan memainkan voice bot
  response_tag = le.inverse_transform([output])[0]
  kata = random.choice(responses[response_tag])
  print("Bot-ol : ", kata)
  tts = gTTS(kata, lang='id')
  tts.save('/content/drive/MyDrive/Chatbot/Model3/Bot-ol.wav')
  time.sleep(0.08)
  ipd.display(ipd.Audio('/content/drive/MyDrive/Chatbot/Model3/Bot-ol.wav', autoplay=True))
  print("="*60 + "\n")
  if response_tag == "goodbye":
    break

Kamu : HAI
1/1 [==============================] - 0s 459ms/step
Bot-ol :  Halo! Saya Bot-ol, salam kenal ya! Mau tau tentang jadwal kuliah, kan?



Kamu : bye
1/1 [==============================] - 0s 22ms/step
Bot-ol :  Dahh, semoga harimu menyenangkan yaa!
